# Создание Hive таблицы через Spark

Этот ноутбук демонстрирует, как создать управляемую таблицу в Hive (через Hive Metastore) с использованием PySpark.

**Предварительные требования:**
- Запущены контейнеры: `spark-master`, `spark-worker`, `hive-metastore`, `hive-server`, `namenode`, `datanode`, `postgres`.

In [1]:
from pyspark.sql import SparkSession

# Создать Spark сессию с поддержкой Hive
# Важно указать spark.hadoop.hive.metastore.uris, чтобы Spark мог связаться с метастором
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("CreateHiveTableApp") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

print("Spark Session created successfully.")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/18 20:02:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/18 20:02:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark Session created successfully.


In [2]:
# Создать тестовые данные
data = [
    (1, "Alice", 25),
    (2, "Bob", 30),
    (3, "Charlie", 35)
]

df = spark.createDataFrame(data, ["id", "name", "age"])

# Показать данные
print("Исходные данные:")
df.show()

Исходные данные:


26/01/18 20:03:04 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
                                                                                

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 25|
|  2|    Bob| 30|
|  3|Charlie| 35|
+---+-------+---+



In [3]:
# Имя базы данных и таблицы
db_name = "default"
table_name = "test_people_table"

# Записать как управляемую (managed) таблицу Hive
# saveAsTable автоматически создает таблицу в Hive Metastore и кладет данные в spark.sql.warehouse.dir
print(f"Сохраняем таблицу {db_name}.{table_name}...")

df.write \
    .mode("overwrite") \
    .saveAsTable(f"{db_name}.{table_name}")

print(f"✓ Таблица {db_name}.{table_name} успешно создана.")

Сохраняем таблицу default.test_people_table...


26/01/18 20:03:26 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


✓ Таблица default.test_people_table успешно создана.


In [4]:
# Проверка: читаем из таблицы через Spark SQL
print("Чтение из созданной таблицы через Spark SQL:")
spark.sql(f"SELECT * FROM {db_name}.{table_name}").show()

Чтение из созданной таблицы через Spark SQL:
+---+-------+---+
| id|   name|age|
+---+-------+---+
|  2|    Bob| 30|
|  3|Charlie| 35|
|  1|  Alice| 25|
+---+-------+---+



In [6]:
spark.stop()